## Hyperparameter tuning

#### Ray tune

![](img/rllib_and_tune.png)

#### Key hypers

- `lr`
- `train_batch_size`
- `sgd_minibatch_size`
- `num_sgd_iter`
- `entropy_coeff`
- model architecture

In [3]:
env_config = {
    "num_candidates" : 2,
    "alpha"          : 0.5,
    "seed"           : 42
}

tune_config = {
    "framework"             : "torch",
    "create_env_on_driver"  : True,
    "seed"                  : 0,
    "lr"                    : 0.001,
    "model"                 : {
        "fcnet_hiddens"     : [64, 64]
    },
    "env"                   : BasicRecommender,
    "env_config"            : env_config
}

In [5]:
tune.run("PPO", config = tune_config, stop = {"training_iteration": 10}, checkpoint_at_end=True, verbose=2, local_dir=".")

(PPOTrainer pid=63452) 2022-07-14 09:51:34,080	INFO ppo.py:249 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
(PPOTrainer pid=63452) 2022-07-14 09:51:34,090	INFO trainer.py:779 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


Trial PPO_BasicRecommender_36008_00000 reported episode_reward_max=26.229951468536488,episode_reward_min=24.437207722145967,episode_reward_mean=25.367144631882343,episode_len_mean=100.0,episode_media={},episodes_this_iter=40,policy_reward_min={},policy_reward_max={},policy_reward_mean={},custom_metrics={},sampler_perf={'mean_raw_obs_processing_ms': 0.03270105622161454, 'mean_inference_ms': 0.25230732516966003, 'mean_action_processing_ms': 0.015700894078869987, 'mean_env_wait_ms': 0.014201394919453113, 'mean_env_render_ms': 0.0},off_policy_estimator={},num_healthy_workers=2,timesteps_this_iter=4000,agent_timesteps_total=4000,timers={'sample_time_ms': 3011.171, 'sample_throughput': 1328.387, 'load_time_ms': 6.652, 'load_throughput': 601333.907, 'learn_time_ms': 1587.199, 'learn_throughput': 2520.163, 'update_time_ms': 2.732},info={'learner': {'default_policy': {'learner_stats': {'allreduce_latency': 0.0, 'cur_kl_coeff': 0.20000000000000004, 'cur_lr': 0.0010000000000000005, 'total_loss': 

Trial PPO_BasicRecommender_36008_00000 reported episode_reward_max=26.52690953227332,episode_reward_min=24.093393864945554,episode_reward_mean=25.429153526172882,episode_len_mean=100.0,episode_media={},episodes_this_iter=40,policy_reward_min={},policy_reward_max={},policy_reward_mean={},custom_metrics={},sampler_perf={'mean_raw_obs_processing_ms': 0.036330457471485056, 'mean_inference_ms': 0.2579944025246475, 'mean_action_processing_ms': 0.015710399918824974, 'mean_env_wait_ms': 0.01514268198918096, 'mean_env_render_ms': 0.0},off_policy_estimator={},num_healthy_workers=2,timesteps_this_iter=4000,agent_timesteps_total=16000,timers={'sample_time_ms': 2459.617, 'sample_throughput': 1626.269, 'load_time_ms': 1.859, 'load_throughput': 2151752.725, 'learn_time_ms': 1446.732, 'learn_throughput': 2764.852, 'update_time_ms': 3.082},info={'learner': {'default_policy': {'learner_stats': {'allreduce_latency': 0.0, 'cur_kl_coeff': 0.20000000000000004, 'cur_lr': 0.0010000000000000005, 'total_loss': 

Trial PPO_BasicRecommender_36008_00000 reported episode_reward_max=26.771151646915843,episode_reward_min=24.011689111219024,episode_reward_mean=25.575060131062806,episode_len_mean=100.0,episode_media={},episodes_this_iter=40,policy_reward_min={},policy_reward_max={},policy_reward_mean={},custom_metrics={},sampler_perf={'mean_raw_obs_processing_ms': 0.0378616620354775, 'mean_inference_ms': 0.263690299528818, 'mean_action_processing_ms': 0.016213090765989046, 'mean_env_wait_ms': 0.015341564616789033, 'mean_env_render_ms': 0.0},off_policy_estimator={},num_healthy_workers=2,timesteps_this_iter=4000,agent_timesteps_total=28000,timers={'sample_time_ms': 2288.985, 'sample_throughput': 1747.5, 'load_time_ms': 1.285, 'load_throughput': 3112573.534, 'learn_time_ms': 1380.818, 'learn_throughput': 2896.834, 'update_time_ms': 2.364},info={'learner': {'default_policy': {'learner_stats': {'allreduce_latency': 0.0, 'cur_kl_coeff': 0.10000000000000002, 'cur_lr': 0.0010000000000000005, 'total_loss': 20.

Trial PPO_BasicRecommender_36008_00000 reported episode_reward_max=26.6548669958362,episode_reward_min=24.056177303270747,episode_reward_mean=25.540941366961775,episode_len_mean=100.0,episode_media={},episodes_this_iter=40,policy_reward_min={},policy_reward_max={},policy_reward_mean={},custom_metrics={},sampler_perf={'mean_raw_obs_processing_ms': 0.036447590817010475, 'mean_inference_ms': 0.25664676157687394, 'mean_action_processing_ms': 0.015811126708607494, 'mean_env_wait_ms': 0.014775917683812385, 'mean_env_render_ms': 0.0},off_policy_estimator={},num_healthy_workers=2,timesteps_this_iter=4000,agent_timesteps_total=40000,timers={'sample_time_ms': 2217.21, 'sample_throughput': 1804.069, 'load_time_ms': 1.0, 'load_throughput': 4000862.307, 'learn_time_ms': 1385.19, 'learn_throughput': 2887.692, 'update_time_ms': 2.919},info={'learner': {'default_policy': {'learner_stats': {'allreduce_latency': 0.0, 'cur_kl_coeff': 0.10000000000000002, 'cur_lr': 0.0010000000000000005, 'total_loss': 20.

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_BasicRecommender_36008_00000,TERMINATED,127.0.0.1:63452,10,23.1234,40000,25.5409,26.6549,24.0562,100


(PPOTrainer pid=63452) 2022-07-14 09:51:57,783	WARNING deprecation.py:45 -- DeprecationWarning: `clear_buffer` has been deprecated. Use `Filter.reset_buffer()` instead. This will raise an error in the future!
2022-07-14 09:51:58,607	INFO tune.py:639 -- Total run time: 27.73 seconds (26.82 seconds for the tuning loop).
